# Emergency Facilities Location Prediction

**Step 1:** Prepare Toronto neigbourhood data

In [2]:
import pandas as pd
import numpy as np
import requests

In [3]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki_url)
codes_df_list = pd.read_html(io = wiki_page.text, attrs = {'class': 'wikitable'})
codes_df_0 = codes_df_list[0]
codes_df_1 = codes_df_0.rename(columns = {'Postcode': 'PostalCode'})
codes_df_2 = codes_df_1[codes_df_1['Borough'] != 'Not assigned']
codes_df_2['Neighbourhood'] = np.where(codes_df_2['Neighbourhood'] == 'Not assigned', codes_df_2['Borough'], codes_df_2['Neighbourhood'])
neighborhood_df = codes_df_2.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda v: ', '.join(v)).reset_index()
neighborhood_df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Step 2:** Prepare emergency statistic data

In [4]:
import urllib
import json

In [16]:
url = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show'
params = { 'id': 'c21f3bd1-e016-4469-abf5-c58bb8e8b5ce'}
response = urllib.request.urlopen(url, data=bytes(json.dumps(params), encoding='utf-8'))
package = json.loads(response.read())
xls_url = package['result']['resources'][0]['url']
print(xls_url)

http://opendata.toronto.ca/it/com/TPS_Incident_Data_2010-2019.xlsx


In [19]:
!wget http://opendata.toronto.ca/it/com/TPS_Incident_Data_2010-2019.xlsx emergency_data.xlsx

--2020-02-10 15:20:12--  http://opendata.toronto.ca/it/com/TPS_Incident_Data_2010-2019.xlsx
Resolving opendata.toronto.ca (opendata.toronto.ca)... 206.130.170.86
Connecting to opendata.toronto.ca (opendata.toronto.ca)|206.130.170.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97534332 (93M)
Saving to: ‘TPS_Incident_Data_2010-2019.xlsx.1’

 0% [                                       ] 957,781      462KB/s             ^C


XLRDError: File size is 0 bytes

In [ ]:
emergency_df = pd.read_excel('emergency_data.xlsx', sheet_name='2010')
emergency_df.head()

In [13]:
emergency_df.shape

(204808, 6)